## Read the config

In [2]:
import yaml

In [3]:
import os
import utils
import torch
import random
import webdataset as wds
# torch.set_float32_matmul_precision('medium')

In [4]:
# # tf32 data type is faster than standard float32
# torch.backends.cuda.matmul.allow_tf32 = True

# ### Multi-GPU config ###
# device_count = torch.cuda.device_count()
# print(f"Number of available CUDA devices: {device_count}")

# local_rank = os.getenv('LOCAL_RANK')
# if local_rank is None: 
#     local_rank = 0
# else:
#     local_rank = int(local_rank)
# print(f"LOCAL RANK={local_rank}")

# num_devices = os.getenv('NUM_GPUS')
# if num_devices is None: 
#     num_devices = 1
# else:
#     num_devices = int(num_devices)
# print(f"NUM GPUS={num_devices}")
# distributed = True if num_devices>1 else False
# if distributed: assert device_count==num_devices

# node = os.getenv('SLURM_NODEID')
# if node is None:
#     node = 0
# else:
#     node = int(node)
# print(f"NODE={node}")

# global_rank = os.getenv('RANK')
# if global_rank is None:
#     global_rank = 0
# else:
#     global_rank = int(global_rank)
# print(f"GLOBAL RANK={global_rank}")

# world_size = os.getenv('WORLD_SIZE')
# if world_size is None: 
#     world_size = 1
# else:
#     world_size = int(world_size)
# print(f"WORLD_SIZE={world_size}")

# if utils.is_interactive():
#     # Following allows you to change functions in models.py or utils.py and 
#     # have this notebook automatically update with your revisions
#     %load_ext autoreload
#     %autoreload 2
#     from tqdm.notebook import tqdm
# else:
#     from tqdm import tqdm

# # Load parameters from yaml config
# config = yaml.load(open('config.yaml', 'r'), Loader=yaml.FullLoader)

# # create global variables from the config
# for attribute_name in config.keys():
#     globals()[attribute_name] = config[f'{attribute_name}']

# data_type = torch.float16 # change depending on your mixed_precision
# # batch_size = global_batch_size // num_devices
# global_batch_size = batch_size * world_size

# # FSDP Setup
# if distributed:
#     import torch.distributed as dist
#     import torch.multiprocessing as mp
#     from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
#     from torch.distributed.fsdp.api import BackwardPrefetch, CPUOffload, ShardingStrategy
#     import functools
#     from torch.distributed.fsdp.wrap import size_based_auto_wrap_policy, transformer_auto_wrap_policy
#     print("starting init_process_group...")
#     dist.init_process_group("nccl", rank=global_rank, world_size=world_size)
#     print(f"setting device to cuda:{local_rank}")
#     try:
#         torch.cuda.set_device(local_rank)
#         device = torch.device('cuda',local_rank)
#         print(f"\nSuccessfully set cuda:{local_rank} | global_rank{global_rank} | node{node}")
#     except Exception as error:        
#         print(f"\nFAILED TO SET DEVICE cuda:{local_rank} | global_rank{global_rank} | node{node}")
#         print("An exception occurred:", error)
        
# else:
#     device = torch.device('cuda')

# print("PID of this process =",os.getpid())
# print("device =", device, "distributed =",distributed, "num_devices =", num_devices, "local rank =", local_rank, "world size =", world_size, "data_type =", data_type)

In [5]:
# Load parameters from yaml config
config = yaml.load(open('config.yaml', 'r'), Loader=yaml.FullLoader)

# create global variables from the config
for attribute_name in config.keys():
    globals()[attribute_name] = config[f'{attribute_name}']

In [6]:
# outdir = os.path.abspath(f'../ckpts/{model_name}')
# os.makedirs(outdir,exist_ok=True)
# print("outdir", outdir)
# print("use_cls_token", use_cls_token)

In [7]:
patch_depth = patch_size[0]
patch_height = patch_size[1]
patch_width = patch_size[2]

## Define the data

In [8]:
# aug_transform = utils.DataPrepper(
#     num_frames=num_frames*2,
#     masking_strategy=masking_strategy,
#     patch_depth=patch_depth,
#     patch_height=patch_height,
#     patch_width=patch_width,
#     frame_patch_size=frame_patch_size,
# )

In [9]:
# def log_and_continue(exn):
#     """Call in an exception handler to ignore any exception, issue a warning, and continue."""
#     print(f'Handling webdataset error ({repr(exn)}). Ignoring.')
#     return True

# def filter_corrupted_images(sample):
#     """If all the required files are not present don't use them."""
#     correct_data = ("func.npy" in sample)
#     return correct_data

# ### ================      Train Dataset and DataLoader    ====================
# from braceexpand import braceexpand
# print(train_urls)
# if is_s3:
#     expanded_urls = [f"pipe:aws s3 cp {url} -" for pattern in train_urls for url in braceexpand(pattern)]
# else:
#     expanded_urls = [str(url) for pattern in train_urls for url in braceexpand(pattern)]

# train_data = (
#     wds.WebDataset(expanded_urls, resampled=True, nodesplitter=wds.split_by_node, handler=log_and_continue)
#     .shuffle(100, initial=100, rng=random.Random(seed))
#     .select(filter_corrupted_images)
#     .decode("torch")
# )
# train_dl = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)

# ### ================      Test Dataset and DataLoader    ====================
# print(test_urls)
# if is_s3:
#     expanded_urls = [f"pipe:aws s3 cp {url} -" for pattern in test_urls for url in braceexpand(pattern)]
# else:
#     expanded_urls = [str(url) for pattern in train_urls for url in braceexpand(pattern)]

# test_data = (
#     wds.WebDataset(expanded_urls, resampled=True, nodesplitter=wds.split_by_node, handler=log_and_continue)
#     .shuffle(100, initial=100, rng=random.Random(seed))
#     .select(filter_corrupted_images)
#     .decode("torch")
# )
# test_dl = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)

In [10]:
# import time
# import numpy as np

# start_time = time.time() 
# num_it = 2
# print(f"Yielding {num_it} batches")

# for i, batch in enumerate(test_dl):
#     print("iter",i)
#     input_func = batch['func.npy']
#     subject_id = batch['subject_id.txt']
#     subject_id = [int(subject[-2:]) for subject in subject_id]
#     # session_id = batch['session_id.txt']
#     # session_id = [int(session[-2:]) for session in session_id]
#     func, brain_pos_pats = aug_transform(input_func)
#     if i >= (num_it-1):
#         break

# print("Done!")
# print("input_func", input_func.shape)
# print("func", func.shape)
# print("subject_id", subject_id)

# end_time = time.time()  
# execution_time = end_time - start_time  
# print(f"Execution time: {execution_time} seconds")

In [11]:
### New datasets
import h5py

In [12]:
from torch.utils.data import Dataset, DataLoader


class fMRIDatasets(Dataset):
    def __init__(self):
        nsd_raw_path = "/weka/proj-medarc/shared/mindeyev2_dataset/"
        f = h5py.File(f'{nsd_raw_path}/subj01_mnidata.h5', 'r') #subj01_rawdata_old.h5
        # mindeye_global_trs = f['global_trs'][:]
        self.data = f['funcs']

    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

In [13]:
# func, brain_pos_pats = aug_transform(input_func)
# print(func.shape)
# display(utils.view_brain(func,cut_coords=(44,44,44)))

In [17]:
train_dataset = fMRIDatasets()
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=512, num_workers=36, drop_last=True) # 42
# test_dataset = fMRIDatasets(mindeye_funcs[-10000:])
# test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=128)

In [ ]:
batch = next(iter(train_dataloader))

In [14]:
# import matplotlib.pyplot as plt

# plt.imshow(func[0,0][:,40,:], cmap='gray')

## Define the model

In [15]:
import torch.nn as nn
from timm.models.vision_transformer import Block

In [16]:
class PatchEmbed(torch.nn.Module):
    def __init__(self, embed_dim, img_size = (88, 104, 72), patch_size = (8,8,6), in_chans: int = 1, bias: bool=True):
        super().__init__()
        self.input_size = img_size
        self.patch_size = patch_size
        self.num_patches = torch.prod(torch.tensor([t1//t2 for t1,t2 in zip(img_size, patch_size)])).item()
        self.proj = nn.Conv3d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size, bias=bias)
    def forward(self, x):
        x = self.proj(x)
        return x.flatten(2).transpose(1, 2)

In [17]:
import torch
import numpy as np

def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=float)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    pos = pos.reshape(-1)  # (M,)
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out) # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)

    emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
    return emb


def get_3d_sincos_pos_embed_from_grid(embed_dim, grid):
    assert embed_dim % 3 == 0

    # use half of dimensions to encode grid_h
    emb_d = get_1d_sincos_pos_embed_from_grid(embed_dim // 3, grid[0])  # (H*W, D/2)
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 3, grid[1])  # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 3, grid[2])  # (H*W, D/2)

    emb = np.concatenate([emb_d, emb_h, emb_w], axis=1) # (H*W, D)
    return emb

def get_3d_sincos_pos_embed(embed_dim, grid_d, grid_h, grid_w, cls_token=False):
    """
    grid_size: int of the grid height and width
    return:
    pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
    """
    grid_h_idx = np.arange(grid_h, dtype=np.float32)
    grid_w_idx = np.arange(grid_w, dtype=np.float32)
    grid_d_idx = np.arange(grid_d, dtype=np.float32)
    
    grid = np.meshgrid(grid_d_idx, grid_h_idx, grid_w_idx)  # here w goes first D, H, W 
    grid = np.stack(grid, axis=0)

    grid = grid.reshape([3, 1, grid_d, grid_h, grid_w])
    pos_embed = get_3d_sincos_pos_embed_from_grid(embed_dim, grid)
    if cls_token:
        pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)
    return pos_embed

In [18]:
# def contrastive_loss(
#     cls_token1: torch.Tensor, cls_token2: torch.Tensor, temperature: float = 0.07, exclude_diagonal: bool = True
# ):
#     feat1 = cls_token1 / cls_token1.norm(dim=1, keepdim=True)
#     feat2 = cls_token2 / cls_token2.norm(dim=1, keepdim=True)

#     cosine_sim = feat1 @ feat2.T / temperature
    
#     # if exclude_diagonal:
#     #     mask = torch.eye(cosine_sim.shape[0], device=cosine_sim.device)
#         # cosine_sim = cosine_sim.masked_fill(mask.bool(), -float('inf'))

#     labels = torch.arange(cosine_sim.shape[0]).to(cosine_sim.device)
#     loss = (
#         torch.nn.functional.cross_entropy(cosine_sim, labels)
#         + torch.nn.functional.cross_entropy(cosine_sim.T, labels)
#     ) / 2
#     return loss

In [19]:
import copy
from typing import Optional, List
import pickle as cp

import torch
import torch.nn.functional as F
from torch import nn, Tensor


class TransformerDecoder(nn.Module):

    def __init__(self, decoder_layer, num_layers, norm=None, return_intermediate=False):
        super().__init__()
        self.layers = _get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm
        self.return_intermediate = return_intermediate

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None):
        output = tgt
        T,B,C = memory.shape
        intermediate = []
        atten_layers = []
        for n,layer in enumerate(self.layers):
   
            residual=True
            output,ws = layer(output, memory,  tgt_mask=tgt_mask,
                           memory_mask=memory_mask,
                           tgt_key_padding_mask=tgt_key_padding_mask,
                           memory_key_padding_mask=memory_key_padding_mask,
                           pos=pos, query_pos=query_pos,residual=residual)
            atten_layers.append(ws)
            if self.return_intermediate:
                intermediate.append(self.norm(output))
        if self.norm is not None:
            output = self.norm(output)
            if self.return_intermediate:
                intermediate.pop()
                intermediate.append(output)

        if self.return_intermediate:
            return torch.stack(intermediate)
        return output,atten_layers

def _get_activation_fn(activation):
    """Return an activation function given a string"""
    if activation == "relu":
        return F.relu
    if activation == "gelu":
        return F.gelu
    if activation == "glu":
        return F.glu
    raise RuntimeError(F"activation should be relu/gelu, not {activation}.")

def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


class TransformerDecoderLayer(nn.Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
        self.normalize_before = normalize_before
    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    def forward_post(self, tgt, memory, 
                     tgt_mask: Optional[Tensor] = None,
                     memory_mask: Optional[Tensor] = None,
                     tgt_key_padding_mask: Optional[Tensor] = None,
                     memory_key_padding_mask: Optional[Tensor] = None,
                     pos: Optional[Tensor] = None,
                     query_pos: Optional[Tensor] = None,
                     residual=True):
        q = k = self.with_pos_embed(tgt, query_pos)
        tgt2,ws = self.self_attn(q, k, value=tgt, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)
        tgt = self.norm1(tgt)
        tgt2,ws = self.multihead_attn(query=self.with_pos_embed(tgt, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)


        # attn_weights [B,NUM_Q,T]
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt,ws

    def forward_pre(self, tgt, memory,
                    tgt_mask: Optional[Tensor] = None,
                    memory_mask: Optional[Tensor] = None,
                    tgt_key_padding_mask: Optional[Tensor] = None,
                    memory_key_padding_mask: Optional[Tensor] = None,
                    pos: Optional[Tensor] = None,
                    query_pos: Optional[Tensor] = None):
        tgt2 = self.norm1(tgt)
        q = k = self.with_pos_embed(tgt2, query_pos)
        tgt2,ws = self.self_attn(q, k, value=tgt2, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)
        tgt = tgt + self.dropout1(tgt2)
        tgt2 = self.norm2(tgt)
        tgt2,attn_weights = self.multihead_attn(query=self.with_pos_embed(tgt2, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)
        tgt = tgt + self.dropout2(tgt2)
        tgt2 = self.norm3(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt2))))
        tgt = tgt + self.dropout3(tgt2)
        return tgt,attn_weights

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None,
                residual=True):
        if self.normalize_before:
            return self.forward_pre(tgt, memory, tgt_mask, memory_mask,
                                    tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos)
        return self.forward_post(tgt, memory, tgt_mask, memory_mask,
                                 tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos,residual)


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class DQNCOSLoss(nn.Module):
    def __init__(self):
        super(DQNCOSLoss, self).__init__()

    def forward(self, input):
        batch_size = input.size(0)
        target = Variable(torch.LongTensor(range(batch_size))).to(input.device)
        loss = 0
        loss += nn.CrossEntropyLoss()(input, target)
        # loss += nn.CrossEntropyLoss()(input.transpose(1, 0), target)
        return loss #/ 2

class Fusion(nn.Module):
    def __init__(self, d_model, n_heads, num_layers, state_prob, dropout=0.1) -> None:
        super().__init__()
        decoder_layer = TransformerDecoderLayer(d_model, n_heads , 1024, 0.1, 'relu',normalize_before=True)
        decoder_norm = nn.LayerNorm(d_model)
        self.decoder = TransformerDecoder(decoder_layer, num_layers , decoder_norm, return_intermediate=False)

        # Learnable Queries
        #self.query_embed = nn.Embedding(config['num_queries'] ,self.d_model)
        self.dropout_feas = nn.Dropout(dropout)

        # Attribute classifier
        self.classifier = nn.Linear(d_model, state_prob)

    def forward(self, query_embed, features):
        features,ws = self.decoder(query_embed, features, 
            memory_key_padding_mask=None, pos=None, query_pos=None)
        out = self.dropout_feas(features)
        out = self.classifier(out).transpose(0,1) #B query Atributes
        return out

In [20]:
class MaskedAutoencoderViT(nn.Module):
    """ Masked Autoencoder with VisionTransformer backbone
    """
    def __init__(self, img_size=(88, 104, 72), patch_size=(8,8,6), in_chans=1,
                 embed_dim=1536, depth=24, num_heads=16,
                 decoder_embed_dim=576, decoder_depth=8, decoder_num_heads=16,
                 mlp_ratio=4., norm_layer=nn.LayerNorm, norm_pix_loss=True):

        super().__init__()

        # --------------------------------------------------------------------------
        self.actual_batch_size = 24
        # MAE encoder specifics
        self.patch_size = patch_size
        self.in_chans = in_chans
        self.img_size = img_size
        self.patch_embed = PatchEmbed(embed_dim, img_size=img_size, patch_size=patch_size, in_chans=in_chans)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim), requires_grad=True)  # fixed sin-cos embedding

        self.blocks = nn.ModuleList([
            Block(embed_dim, num_heads, mlp_ratio, qkv_bias=True, norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)
        # --------------------------------------------------------------------------

        # --------------------------------------------------------------------------
        # MAE decoder specifics
        self.decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias=True)

        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))

        self.decoder_pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, decoder_embed_dim), requires_grad=True)  # fixed sin-cos embedding

        self.decoder_blocks = nn.ModuleList([
            Block(decoder_embed_dim, decoder_num_heads, mlp_ratio, qkv_bias=True, norm_layer=norm_layer)
            for i in range(decoder_depth)])

        self.decoder_norm = norm_layer(decoder_embed_dim)
        self.decoder_pred = nn.Linear(decoder_embed_dim, patch_size[0] * patch_size[1] * patch_size[2] * in_chans, bias=True) # decoder to patch
        # --------------------------------------------------------------------------
        # MAE Fusion module
        self.fusion_module = Fusion(d_model=embed_dim, num_layers=4, n_heads=4, state_prob=1, dropout=0.1) # base fusion module params are from the paper
        self.ce_loss = DQNCOSLoss()
        
        self.norm_pix_loss = norm_pix_loss

        self.initialize_weights()

    def initialize_weights(self):
        # initialization
        # initialize (and freeze) pos_embed by sin-cos embedding
        num_patches_d = self.img_size[0]//self.patch_size[0]
        num_patches_h = self.img_size[1]//self.patch_size[1]
        num_patches_w = self.img_size[2]//self.patch_size[2]
        pos_embed = get_3d_sincos_pos_embed(self.pos_embed.shape[-1], num_patches_d, num_patches_h, num_patches_w, cls_token=True)
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))

        decoder_pos_embed = get_3d_sincos_pos_embed(self.decoder_pos_embed.shape[-1], num_patches_d, num_patches_h, num_patches_w, cls_token=True)
        self.decoder_pos_embed.data.copy_(torch.from_numpy(decoder_pos_embed).float().unsqueeze(0))

        # initialize patch_embed like nn.Linear (instead of nn.Conv2d)
        w = self.patch_embed.proj.weight.data
        torch.nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

        # timm's trunc_normal_(std=.02) is effectively normal_(std=0.02) as cutoff is too big (2.)
        torch.nn.init.normal_(self.cls_token, std=.02)
        torch.nn.init.normal_(self.mask_token, std=.02)

        # initialize nn.Linear and nn.LayerNorm
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            # we use xavier_uniform following official JAX ViT:
            torch.nn.init.xavier_uniform_(m.weight)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def patchify(self, imgs):
        """
        imgs: (N, 1, D, H, W)
        x: (N, L, p1 * p2 * p3 * 1)
        """

        N, C, D, H, W = imgs.shape
        p_d, p_h, p_w = self.patch_size
        d, h, w = D//p_d, H//p_h, W//p_w
        
        x = imgs.reshape(shape=(N, C, d, p_d, h, p_h, w, p_w))
        x = torch.einsum("nctuhpwq->nthwupqc", x)
        x = x.reshape(shape=(N, d * h * w, p_d * p_h * p_w * C))
        self.patch_info = (N, C, D, H, W, p_d, p_h, p_w, d, h, w)
        
        return x

    def unpatchify(self, x):
        """
        x: (N, L, p1 * p2 * p3 * 1)
        imgs: (N, 1, D, H, W)

        x: (N, L, patch_size**2 *3)
        imgs: (N, 3, H, W)
        """
        N, C, D, H, W, p_d, p_h, p_w, d, h, w = self.patch_info
        x = x.reshape(shape=(N, d, h, w, p_d, p_h, p_w, C))
        x = torch.einsum("nthwupqc->nctuhpwq", x)
        imgs = x.reshape(shape=(N, C, D, H, W))
        return imgs

    def random_masking(self, x, mask_ratio):
        """
        Perform per-sample random masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        N, L, D = x.shape  # batch, length, dim
        len_keep = int(L * (1 - mask_ratio))
        
        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]
        
        # sort noise for each sample
        ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
        ids_restore = torch.argsort(ids_shuffle, dim=1)

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]
        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

        # generate the binary mask: 0 is keep, 1 is remove
        mask = torch.ones([N, L], device=x.device)
        mask[:, :len_keep] = 0
        # unshuffle to get the binary mask
        mask = torch.gather(mask, dim=1, index=ids_restore)

        return x_masked, mask, ids_restore

    def forward_encoder(self, x, mask_ratio):
        # embed patches
        x = self.patch_embed(x)

        # add pos embed w/o cls token
        x = x + self.pos_embed[:, 1:, :]

        # masking: length -> length * mask_ratio
        x, mask, ids_restore = self.random_masking(x, mask_ratio)

        # append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        return x, mask, ids_restore

    def forward_encoder_latent(self, x, mask_ratio):
        x = self.patch_embed(x)
        x = x + self.pos_embed[:, 1:, :]
        
        
        N, L, D = x.shape  # batch, length, dim
        len_keep = int(L * (1 - mask_ratio))
        noise = torch.rand(N, L, device=x.device)
        ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]
        x = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        return x
        

    def forward_decoder(self, x, ids_restore):
        # embed tokens
        x = self.decoder_embed(x)

        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1)
        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim=1)  # no cls token
        x_ = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2]))  # unshuffle
        x = torch.cat([x[:, :1, :], x_], dim=1)  # append cls token

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        x = self.decoder_pred(x)

        # remove cls token
        x = x[:, 1:, :]

        return x

    def forward_loss(self, imgs, pred, mask):
        """
        imgs: [N, 3, H, W]
        pred: [N, L, p*p*3]
        mask: [N, L], 0 is keep, 1 is remove, 
        """
        target = self.patchify(imgs)
        if self.norm_pix_loss:
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            target = (target - mean) / (var + 1.e-6)**.5

        loss = (pred - target) ** 2
        loss = loss.mean(dim=-1)  # [N, L], mean loss per patch

        loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
        return loss

    # def forward(self, imgs, mask_ratio=0.85):
    #     latent1, mask1, ids_restore1 = self.forward_encoder(imgs, mask_ratio)
    #     # latent2, mask2, ids_restore2 = self.forward_encoder(imgs, mask_ratio)
    #     pred1 = self.forward_decoder(latent1, ids_restore1)  # [N, L, p*p*3]
    #     # pred2 = self.forward_decoder(latent2, ids_restore2)  # [N, L, p*p*3]

    #     # cls_token1 = latent1[:, 0]
    #     # cls_token2 = latent2[:, 0]
        
        
    #     loss = self.forward_loss(imgs, pred1, mask1) # + self.forward_loss(imgs, pred2, mask2)
    #     # loss +=  contrastive_loss(cls_token1, cls_token2) / 2
        
    #     return loss, pred1, mask1

    def forward(self, imgs, mask_ratio=0.85):
        # latent1, mask1, ids_restore1 = self.forward_encoder(imgs, mask_ratio)
        # latent2, mask2, ids_restore2 = self.forward_encoder(imgs, mask_ratio)

        # # recon loss
        # pred1 = self.forward_decoder(latent1, ids_restore1)  # [N, L, p*p*3]
        # loss_recon_1 = self.forward_loss(imgs, pred1, mask1)
        # pred2 = self.forward_decoder(latent2, ids_restore2)  # [N, L, p*p*3]
        # loss_recon_2 = self.forward_loss(imgs, pred2, mask2)

        # # contrastive loss
        # latent1_g = latent1[:, 0, ...]
        # latent2_g = latent2[:, 0, ...]

        # latent1_l = latent1[:, 1:, ...]
        # latent2_l = latent2[:, 1:, ...]

        # l1_2_l2_cls = self.fusion_module(latent1.permute(1, 0, 2), latent2[:, 0, :].unsqueeze(0)).squeeze(-1)
        # l2_2_l1_cls = self.fusion_module(latent2.permute(1, 0, 2), latent1[:, 0, :].unsqueeze(0)).squeeze(-1)

        # ce_loss0 = self.ce_loss(l1_2_l2_cls)
        # ce_loss1 = self.ce_loss(l2_2_l1_cls)

        # loss = loss_recon_1 + loss_recon_2 + ce_loss0/2 + ce_loss1/2

        ## Large batch size
        latent1, mask1, ids_restore1 = self.forward_encoder(imgs[:self.actual_batch_size, ...], mask_ratio)
        latent2, mask2, ids_restore2 = self.forward_encoder(imgs[:self.actual_batch_size, ...], mask_ratio)

        # recon loss
        pred1 = self.forward_decoder(latent1, ids_restore1)  # [N, L, p*p*3]
        loss_recon_1 = self.forward_loss(imgs, pred1, mask1)
        pred2 = self.forward_decoder(latent2, ids_restore2)  # [N, L, p*p*3]
        loss_recon_2 = self.forward_loss(imgs, pred2, mask2)

        # contrastive loss
        with torch.no_grad():
            latent1_ng = self.forward_encoder_latent(imgs[self.actual_batch_size:, ...], mask_ratio)
            latent2_ng = self.forward_encoder_latent(imgs[self.actual_batch_size:, ...], mask_ratio)
        latent1 = torch.cat([latent1, latent1_ng])
        latent2 = torch.cat([latent2, latent2_ng])
        
        latent1_g = latent1[:, 0, ...]
        latent2_g = latent2[:, 0, ...]

        latent1_l = latent1[:, 1:, ...]
        latent2_l = latent2[:, 1:, ...]

        l1_2_l2_cls = self.fusion_module(latent1.permute(1, 0, 2), latent2[:, 0, :].unsqueeze(0)).squeeze(-1)
        l2_2_l1_cls = self.fusion_module(latent2.permute(1, 0, 2), latent1[:, 0, :].unsqueeze(0)).squeeze(-1)

        ce_loss0 = self.ce_loss(l1_2_l2_cls)
        ce_loss1 = self.ce_loss(l2_2_l1_cls)

        loss = loss_recon_1 + loss_recon_2 + ce_loss0/2 + ce_loss1/2

        # loss = loss_recon_1 + loss_recon_2
        
        return loss, pred1, mask1

    def forward_inference(self, imgs, mask_ratio=0.75):
        latent, mask, ids_restore = self.forward_encoder(imgs, mask_ratio)
        pred = self.forward_decoder(latent, ids_restore)  # [N, L, p*p*3]
        if self.norm_pix_loss:
            target = self.patchify(imgs)
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            pred = (pred * (var + 1.e-6)**.5) + mean
            
        loss = self.forward_loss(imgs, pred, mask)
        return loss, pred, mask

In [21]:
# 576, 14
model = MaskedAutoencoderViT(depth=16, decoder_embed_dim=576, embed_dim=1056, decoder_num_heads=8)
model.patch_embed.num_patches

1716

In [22]:
# ## use large batch size
# actial_batch_size = 24
# mask_ratio = 0.75
# imgs = torch.randn(1024, 1, 88, 104, 72)
# device = torch.device("cuda")
# self = model
# self.to(device)

# latent1, mask1, ids_restore1 = self.forward_encoder(imgs[:actial_batch_size, ...].to(device), mask_ratio)
# latent2, mask2, ids_restore2 = self.forward_encoder(imgs[:actial_batch_size, ...].to(device), mask_ratio)

In [23]:
# with torch.no_grad():
#     latent1_ng = self.forward_encoder_latent(imgs[actial_batch_size:, ...].to(device), 0.75)
#     latent2_ng = self.forward_encoder_latent(imgs[actial_batch_size:, ...].to(device), 0.75)

In [24]:
# latent1.shape, latent1_ng.shape

In [25]:
# # # ## load the previous model state dict
# ckpt = torch.load("/weka/proj-fmri/souvik/fMRI-foundation-model/ckpts/mindeye_ds_exp_1_gl/epoch=32-train_loss=6.22782.ckpt", map_location=torch.device('cpu'))

In [26]:
# state_dict = ckpt["state_dict"]
# state_dict = {k.replace('model.', ""): v for k,v in state_dict.items()}
# model.load_state_dict(state_dict)

In [27]:
# batch = next(iter(train_dataloader))
# # input_func = batch
# # func, _ = aug_transform(input_func)
# # func = func[:, random.randint(0,7), ...].unsqueeze(1)
# func = batch.unsqueeze(1)
# func.shape

In [28]:
# device = torch.device("cuda")
# model.to(device)
# _, pred, mask = model.forward_inference(func.float().to(device))

In [29]:
# patched_imgs = model.patchify(func).to(device)
# patched_imgs = patched_imgs * (~mask.unsqueeze(-1).bool())

In [30]:
# unpatchify = model.unpatchify((pred* mask.unsqueeze(-1)) + patched_imgs)

In [31]:
# import matplotlib.pyplot as plt


# for i in range(5):
#     plt.figure()
#     plt.subplot(1,3,1)
#     # plt.title("pred")
#     plt.imshow(unpatchify[i,0,:, 90, ...].detach().cpu().numpy(), cmap=plt.cm.bone)
#     plt.subplot(1,3,2)
#     # plt.title("gt")
#     plt.imshow(func[i,0,:, 90, ...], cmap=plt.cm.bone)
#     plt.subplot(1,3,3)
#     plt.imshow(model.unpatchify(patched_imgs).detach().cpu().numpy()[i,0,:, 90, ...], cmap=plt.cm.bone)

In [32]:
# import matplotlib.pyplot as plt

# plt.imshow(unpatchify[0,0,:, 30, ...].detach().cpu().numpy(), cmap=plt.cm.bone)

In [33]:
# plt.imshow(func[0,0,:, 90, ...], cmap=plt.cm.bone)

In [34]:
# ## check the embeddings

# with torch.no_grad():
#     x = model.patch_embed(func.to(device))
#     x = x + model.pos_embed[:, 1:, :]
#     cls_token = model.cls_token + model.pos_embed[:, :1, :]
#     cls_tokens = cls_token.expand(x.shape[0], -1, -1)
#     x = torch.cat((cls_tokens, x), dim=1)
#     for blk in model.blocks:
#         x = blk(x)
#     x = model.norm(x)

# logits = x[:, 0, ...]

In [35]:
# torch.nn.functional.cosine_similarity(logits[0].unsqueeze(0), logits[8].unsqueeze(0))

In [36]:
# import lightning as L
# class TrainerfMRIFM(L.LightningModule):
#     def __init__(self, model, total_steps, masking_ratio = 0.75, lr = 1.0e-4):
#         super().__init__()
#         self.model = model
#         self.total_steps = total_steps
#         self.lr = lr
#         self.masking_ratio = masking_ratio
        
#     def forward(self, x):
#         loss, _, _ = self.model(x, self.masking_ratio)
#         return loss

#     def training_step(self, batch, batch_idx):
#         input_func = batch['func.npy']
#         func, _ = aug_transform(input_func)
#         func = func[:, random.randint(0,7), ...].unsqueeze(1)
#         loss = self(func)
#         self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=len(batch), sync_dist=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         input_func = batch['func.npy']
#         func, _ = aug_transform(input_func)
#         func = func[:, random.randint(0,7), ...].unsqueeze(1)
#         loss = self(func)
#         self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=len(batch), sync_dist=True)
#         return loss

#     def configure_optimizers(self):
#         self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
#         self.lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=self.lr, total_steps=self.total_steps)
#         return {"optimizer": self.optimizer, "lr_scheduler": {
#             "scheduler": self.lr_scheduler,
#             "interval": "step",
#             "frequency": 1
#         }}

    

In [37]:
# # num_epochs = 10
# # num_samples_per_epoch = 20
# # batch_size = 4

# max_steps = (num_epochs * num_samples_per_epoch)//batch_size
# eval_per_n_steps = (num_samples_per_epoch//batch_size)
# limit_val_batches = (test_num_samples_per_epoch//batch_size)
# max_steps, eval_per_n_steps

In [38]:
# from lightning import Trainer
# from lightning.pytorch.loggers import WandbLogger
# from lightning.pytorch.strategies import DDPStrategy
# from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint, RichModelSummary, RichProgressBar

# checkpoint_callback = ModelCheckpoint(
#     dirpath=os.path.abspath(f'../ckpts/{model_name}'),
#     filename='{epoch}-{val_loss:.5f}',
#     save_top_k=1,
#     verbose=True,
#     monitor='val_loss',
#     mode='min',
# )
# callbacks = [
#     LearningRateMonitor(),
#     checkpoint_callback,
#     # RichModelSummary(max_depth=2),
#     # RichProgressBar()
# ]
# trainer = Trainer(
#     devices="auto",
#     num_nodes=int(os.getenv('NUM_NODES', 1)),
#     precision="16-mixed",
#     logger=WandbLogger(project='mae_single_tf', id=model_name, name=model_name),
#     callbacks=callbacks,
#     max_steps=max_steps,
#     # strategy = DDPStrategy(process_group_backend="gloo"), # COMMENT THIS IF USING NOTEBOOK
#     val_check_interval=eval_per_n_steps,
#     log_every_n_steps=5,
#     limit_val_batches=limit_val_batches,
#     limit_train_batches = eval_per_n_steps,
#     accumulate_grad_batches=1
# )

In [39]:
# module = TrainerfMRIFM(model=model, total_steps=max_steps, masking_ratio=0.8, lr=max_lr)

In [40]:
# trainer.fit(module, train_dataloaders=train_dl, val_dataloaders=test_dl)

In [41]:
import lightning as L
class TrainerfMRIFM(L.LightningModule):
    def __init__(self, model, total_steps, masking_ratio = 0.75, lr = 1.0e-4):
        super().__init__()
        self.model = model
        self.total_steps = total_steps
        self.lr = lr
        self.masking_ratio = masking_ratio
        
    def forward(self, x):
        loss, _, _ = self.model(x, self.masking_ratio)
        return loss

    def training_step(self, batch, batch_idx):
        batch = batch.unsqueeze(1)
        loss = self(batch)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=len(batch), sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        batch = batch.unsqueeze(1)
        loss = self(batch)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=len(batch), sync_dist=True)
        return loss

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        self.lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=self.lr, total_steps=self.total_steps)
        return {"optimizer": self.optimizer, "lr_scheduler": {
            "scheduler": self.lr_scheduler,
            "interval": "step",
            "frequency": 1
        }}

num_epochs = 100
max_steps = (num_epochs * len(train_dataloader))

from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.strategies import DDPStrategy
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint, RichModelSummary, RichProgressBar

checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.abspath(f'../ckpts/{model_name}'),
    filename='{epoch}-{train_loss:.5f}',
    save_top_k=1,
    verbose=True,
    monitor='train_loss',
    mode='min',
)
callbacks = [
    LearningRateMonitor(),
    checkpoint_callback,
    # RichModelSummary(max_depth=2),
    # RichProgressBar()
]
trainer = Trainer(
    devices="auto",
    num_nodes=int(os.getenv('NUM_NODES', 1)),
    precision="16-mixed",
    logger=WandbLogger(project='mae_single_tf', id=model_name, name=model_name),
    callbacks=callbacks,
    max_steps=max_steps,
    # strategy = DDPStrategy(process_group_backend="gloo"), # COMMENT THIS IF USING NOTEBOOK
    # val_check_interval=eval_per_n_steps,
    log_every_n_steps=5,
    # limit_val_batches=limit_val_batches,
    # limit_train_batches = eval_per_n_steps,
    accumulate_grad_batches=1
)


/weka/proj-fmri/souvik/fMRI-foundation-model/found/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.10 /weka/proj-fmri/souvik/fMRI-foundation-model/fou ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
module = TrainerfMRIFM(model=model, total_steps=max_steps, masking_ratio=0.75, lr=max_lr)
trainer.fit(module, train_dataloaders=train_dataloader)

/weka/proj-fmri/souvik/fMRI-foundation-model/found/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: mandalsouvik. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [6]

  | Name  | Type                 | Params | Mode 
-------------------------------------------------------
0 | model | MaskedAutoencoderViT | 294 M  | train
-------------------------------------------------------
294 M     Trainable params
0         Non-trainable params
294 M     Total params
1,178.744 Total estimated model params size (MB)


Training: |                                                                                                   …

In [ ]:
# batch = next(iter(train_dataloader))
# batch.shape

In [ ]:
# model.patch_embed #(batch)

## Linear probe 

In [ ]:
# ## load the previous model state dict
# ckpt = torch.load("/weka/proj-fmri/souvik/fMRI-foundation-model/ckpts/base_exp_223m/epoch=6-val_loss=0.24703.ckpt", map_location=torch.device('cpu'))

# state_dict = ckpt["state_dict"]
# state_dict = {k.replace('model.', ""): v for k,v in state_dict.items()}
# model.load_state_dict(state_dict)

In [ ]:
# import lightning as L
# from torchmetrics import Accuracy

In [ ]:
# class fMRIFMLP(L.LightningModule):
#     def __init__(self, model, total_steps, lr = 1.0e-4, embed_dim=1056,num_classes=8):
#         super().__init__()
        
#         self.model = model
#         self.model.eval()
#         for param in self.model.parameters():
#             param.requires_grad = False
        
#         self.total_steps = total_steps
#         self.lr = lr

#         self.head = nn.Sequential(
#             nn.Linear(embed_dim, num_classes),
#         )
#         self.loss = nn.CrossEntropyLoss()
#         self.metric = nn.ModuleDict({
#             "train_metric": Accuracy(task="multiclass", num_classes=num_classes),
#             "val_metric": Accuracy(task="multiclass", num_classes=num_classes)
#         })
        
#     def forward(self, x):
#         with torch.no_grad():
#             x = self.model.patch_embed(x)
#             x = x + self.model.pos_embed[:, 1:, :]
#             cls_token = self.model.cls_token + self.model.pos_embed[:, :1, :]
#             cls_tokens = cls_token.expand(x.shape[0], -1, -1)
#             x = torch.cat((cls_tokens, x), dim=1)
#             for blk in self.model.blocks:
#                 x = blk(x)
#             x = self.model.norm(x)

#         logits = self.head(x[:, 0, ...])
        
#         return logits
#     def shared_step(self, batch, phase):
#         input_func = batch['func.npy']
#         func, _ = aug_transform(input_func)
#         func = func[:, 0, ...].unsqueeze(1)

#         subject_id = batch['subject_id.txt']
#         subject_id = torch.Tensor([int(subject[-2:]) for subject in subject_id]).long().to(self.device) - 1
        
#         logits = self(func)
#         loss = nn.functional.cross_entropy(logits, subject_id)
#         self.metric[phase+ "_metric"](logits, subject_id)
#         self.log(f"{phase}_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=len(batch), sync_dist=True)
#         self.log(f"{phase}_acc", self.metric[phase+ "_metric"], on_step=False, on_epoch=True, prog_bar=True, logger=True, batch_size=len(batch), sync_dist=True)
        
#         return loss

#     def training_step(self, batch, batch_idx):
#         return self.shared_step(batch, "train")

#     def validation_step(self, batch, batch_idx):
#         return self.shared_step(batch, "val")

#     def configure_optimizers(self):
#         self.optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
#         self.lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=self.lr, total_steps=self.total_steps)
#         return {"optimizer": self.optimizer, "lr_scheduler": {
#             "scheduler": self.lr_scheduler,
#             "interval": "step",
#             "frequency": 1
#         }}

In [ ]:
# num_samples_per_epoch = 2048
# test_num_samples_per_epoch = 2048

# max_steps = (num_epochs * num_samples_per_epoch)//batch_size
# eval_per_n_steps = (num_samples_per_epoch//batch_size)
# limit_val_batches = (test_num_samples_per_epoch//batch_size)
# print(max_steps, eval_per_n_steps)

In [ ]:
# model_lp = fMRIFMLP(model=model, total_steps=max_steps)

In [ ]:
# from lightning import Trainer
# from lightning.pytorch.loggers import WandbLogger
# from lightning.pytorch.strategies import DDPStrategy
# from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint, RichModelSummary, RichProgressBar


# checkpoint_callback = ModelCheckpoint(
#     dirpath=os.path.abspath(f'../ckpts/{model_name}'),
#     filename='{epoch}-{val_loss:.5f}',
#     save_top_k=1,
#     verbose=True,
#     monitor='val_loss',
#     mode='min',
# )
# callbacks = [
#     LearningRateMonitor(),
#     # checkpoint_callback,
#     # RichModelSummary(max_depth=2),
#     # RichProgressBar()
# ]
# trainer = Trainer(
#     devices="auto",
#     num_nodes=int(os.getenv('NUM_NODES', 1)),
#     precision="16-mixed",
#     logger=WandbLogger(project='mae_single_tf', id=model_name+"_linear", name=model_name+"_linear"),
#     callbacks=callbacks,
#     max_steps=max_steps,
#     # strategy = DDPStrategy(process_group_backend="gloo"), # COMMENT THIS IF USING NOTEBOOK
#     val_check_interval=eval_per_n_steps,
#     log_every_n_steps=5,
#     limit_val_batches=limit_val_batches,
#     limit_train_batches = eval_per_n_steps,
#     accumulate_grad_batches=1
# )

In [ ]:
# trainer.fit(model_lp, train_dataloaders=train_dl, val_dataloaders=test_dl)

In [ ]:
# trainer.save_checkpoint("/weka/proj-fmri/souvik/fMRI-foundation-model/ckpts/base_exp_223m_linear/trainer_weights.ckpt")